In [10]:
import numpy as np 
import pandas as pd


In [ ]:
df = pd.read_csv("edu_melted.csv")
df2=df[df['year']>2019]

In [6]:
complete_categories = (
    df.groupby("Country Code")["year"]
      .nunique()
      .eq(df["year"].nunique())
      .sum()
)

In [7]:
complete_categories

266

In [11]:
prp = pd.read_csv("WD_PRP_Projections.csv")

# See all education level labels
sorted(prp["education"].unique())

['Bachelor',
 'Incomplete Primary',
 'Lower Secondary',
 'Master and higher',
 'No Education',
 'Post Secondary',
 'Primary',
 'Short Post Secondary',
 'Upper Secondary']

In [12]:
# Tertiary / post-secondary categories
TERTIARY_LEVELS = [
    "Bachelor",
    "Master and higher",
    "Post Secondary",
    "Short Post Secondary"
]

# Filter only rows with tertiary education
tertiary = prp[prp["education"].isin(TERTIARY_LEVELS)]

# Keep youth ages
youth_tertiary = tertiary[tertiary["age"].isin(["15-19", "20-24"])]

# Compute % of youth with tertiary (simple average)
tertiary_share = (
    youth_tertiary.groupby(["iso3", "year"])["prop"]
    .sum()         # sum because these are multiple tertiary categories
    .reset_index()
    .rename(columns={"prop": "tertiary_percent"})
)

print(tertiary_share.head())

  iso3  year  tertiary_percent
0  ABW  2025            176.26
1  ABW  2030            187.63
2  ABW  2035            198.23
3  AFG  2025            144.51
4  AFG  2030            178.40


In [13]:
# Question 1

pop = pd.read_csv("UN_Pop_Projections.csv")
mys = pd.read_csv("WD_MYS_Projections.csv")
prp = pd.read_csv("WD_PRP_Projections.csv")

In [15]:
# -------------------------
# 2. FILTER YOUTH AGES
# -------------------------
YOUTH_AGES_POP = ["15-19", "20-24"]
YOUTH_AGES_EDU = ["15-19", "20-24"]

pop_youth = pop[pop["age"].isin(YOUTH_AGES_POP)]
mys_youth = mys[mys["age"].isin(YOUTH_AGES_EDU)]
prp_youth = prp[prp["age"].isin(YOUTH_AGES_EDU)]

In [16]:
# -------------------------
# 3. CREATE INCOME GROUP LABEL
# (LIC, LMC, UMC, HIC)
# -------------------------
def assign_income_group(row):
    for g in ["LIC", "LMC", "UMC", "HIC"]:
        if row[g] == True:
            return g
    return None

pop_youth["income_group"] = pop_youth.apply(assign_income_group, axis=1)
mys_youth["income_group"] = mys_youth.apply(assign_income_group, axis=1)
prp_youth["income_group"] = prp_youth.apply(assign_income_group, axis=1)

In [17]:
# -------------------------
# 4. YOUTH POPULATION GROWTH 2025 → 2035
# -------------------------
pop_summary = (
    pop_youth.groupby(["income_group", "year"])["population"]
    .sum()
    .reset_index()
)

pop_2025 = pop_summary[pop_summary["year"] == 2025]
pop_2035 = pop_summary[pop_summary["year"] == 2035]

pop_growth = pop_2025.merge(pop_2035, on="income_group", suffixes=("_2025", "_2035"))
pop_growth["youth_pop_growth_pct"] = (
    (pop_growth["population_2035"] - pop_growth["population_2025"])
    / pop_growth["population_2025"]
) * 100

In [18]:
# -------------------------
# 5. MYS GROWTH 2025 → 2035
# -------------------------
mys_summary = (
    mys_youth.groupby(["income_group", "year"])["mys"]
    .mean()
    .reset_index()
)

mys_2025 = mys_summary[mys_summary["year"] == 2025]
mys_2035 = mys_summary[mys_summary["year"] == 2035]

mys_growth = mys_2025.merge(mys_2035, on="income_group", suffixes=("_2025", "_2035"))
mys_growth["mys_growth"] = mys_growth["mys_2035"] - mys_growth["mys_2025"]

EDUCATION PRESSURE INDEX:
How fast the youth population is growing relative to how fast education levels (MYS) are improving.

In [19]:
# -------------------------
# 7. EDUCATION PRESSURE INDEX
# -------------------------
final = pop_growth.merge(mys_growth, on="income_group")
final["education_pressure_index"] = (
    final["youth_pop_growth_pct"] / final["mys_growth"]
)


final = final.drop(columns=[col for col in final.columns if col.startswith("year_")])

print(final)

  income_group  population_2025  population_2035  youth_pop_growth_pct  \
0          HIC     4.240785e+06     4.214800e+06             -0.612755   
1          LIC     3.356140e+06     4.205648e+06             25.312078   
2          LMC     1.500420e+07     1.572809e+07              4.824551   
3          UMC     9.855159e+06     1.012037e+07              2.691117   

    mys_2025   mys_2035  mys_growth  education_pressure_index  
0  10.943259  10.912134   -0.031124                 19.687294  
1   6.894987   7.341880    0.446893                 56.640088  
2   8.930367   9.177820    0.247453                 19.496813  
3  10.476813  10.533487    0.056673                 47.484708  


LICs have the highest pressure

Youth population exploding

Education improving slowly
→ System overwhelmed
→ growing skills gap

2. LMCs have moderate pressure

Youth growing, but manageable

Education improving faster than LICs
→ Balanced growth (Scenario B)

3. UMCs have high pressure for a different reason

Youth barely grows

But MYS barely increases either
→ Education stagnation slows skill upgrading (Scenario C)

4. HICs have low structural pressure

Youth declining

Education saturated

In [20]:
import plotly.express as px
import plotly.graph_objects as go

In [22]:
fig = px.bar(
    final,
    x="income_group",
    y="youth_pop_growth_pct",
    title="Youth Population Growth (2025 → 2035)",
    labels={"income_group": "Income Group", "youth_pop_growth_pct": "Youth Population Growth (%)"},
)

fig.update_traces(
    text=[f"{v:.2f}%" for v in final["youth_pop_growth_pct"]],
    textposition='outside'
)

fig.show()


In [24]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=final["income_group"],
    y=final["youth_pop_growth_pct"],
    name="Youth Pop Growth (%)",
    marker_color="steelblue"
))

fig.add_trace(go.Scatter(
    x=final["income_group"],
    y=final["mys_growth"],
    name="MYS Growth (years)",
    mode="lines+markers",
    marker_color="darkorange",
    yaxis="y2"
))

fig.update_layout(
    title="Population Pressure vs Education Improvement",
    yaxis=dict(title="Youth Pop Growth (%)"),
    yaxis2=dict(title="MYS Growth (years)", overlaying="y", side="right")
)

fig.show()


In [28]:
fig = go.Figure()

for i, row in final.iterrows():
    fig.add_trace(go.Scatter(
        x=[row["mys_2025"], row["mys_2035"]],
        y=[row["income_group"], row["income_group"]],
        mode="lines+markers",
        marker=dict(size=14),
        line=dict(width=4),
        name=row["income_group"]
    ))

fig.update_layout(
    title="Mean Years of Schooling: 2025 vs 2035 (Dumbbell Chart)",
    xaxis_title="Mean Years of Schooling (MYS)",
    yaxis_title="Income Group",
    template="plotly_white"
)

fig.show()


In [38]:
gdp = pd.read_csv("avg_gdp_income_groups_2025_2035.csv")

In [39]:
gdp = gdp.rename(columns={'income_classification': 'income_group'})
print("\nDataFrame after renaming a single column:")
print(gdp)


DataFrame after renaming a single column:
  income_group  variable  value_projected
0          HIC      2025     47487.757640
1          HIC      2035     51542.501300
2          LIC      2025       788.755455
3          LIC      2035       755.642500
4          LMC      2025      2545.443939
5          LMC      2035      2381.028413
6          UMC      2025      8326.312297
7          UMC      2035      8488.777600


In [42]:
# Pivot GDP into columns
gdp_pivot = gdp.pivot(
    index="income_group",
    columns="variable",
    values="value_projected"
).reset_index()

# Rename pivot columns
gdp_pivot = gdp_pivot.rename(columns={
    2025: "gdp_2025",
    2035: "gdp_2035"
})

In [43]:
# Merge GDP into your final table
final = final.merge(gdp_pivot, on="income_group", how="left")

# Calculate GDP growth %
final["gdp_growth_pct"] = (final["gdp_2035"] - final["gdp_2025"]) / final["gdp_2025"] * 100

final

,income_group,population_2025,population_2035,youth_pop_growth_pct,mys_2025,mys_2035,mys_growth,education_pressure_index,gdp_2025_x,gdp_2035_x,gdp_growth_pct,gdp_2025_y,gdp_2035_y,gdp_2025,gdp_2035
0,HIC,4.240785e+06,4.214800e+06,-0.612755,10.943259,10.912134,-0.031124,19.687294,47487.757640,51542.5013,8.538503,47487.757640,51542.501300,47487.757640,51542.501300
1,LIC,3.356140e+06,4.205648e+06,25.312078,6.894987,7.341880,0.446893,56.640088,788.755455,755.6425,-4.198127,788.755455,755.642500,788.755455,755.642500
2,LMC,1.500420e+07,1.572809e+07,4.824551,8.930367,9.177820,0.247453,19.496813,NaN,NaN,-6.459208,2545.443939,2381.028413,2545.443939,2381.028413
3,UMC,9.855159e+06,1.012037e+07,2.691117,10.476813,10.533487,0.056673,47.484708,NaN,NaN,1.951228,8326.312297,8488.777600,8326.312297,8488.777600


In [46]:
fig = px.scatter(
    final,
    x="mys_growth",
    y="gdp_growth_pct",
    color="income_group",
    text="income_group",
    title="Quadrant View: MYS Growth vs GDP Growth"
)

# add quadrant lines
fig.add_vline(x=0, line_dash="dash", line_color="gray")
fig.add_hline(y=0, line_dash="dash", line_color="gray")

fig.update_traces(textposition="top center")
fig.update_layout(template="plotly_white")

fig.show()

In [47]:
df_g = final[["income_group", "gdp_2025", "gdp_2035"]].melt(
    id_vars="income_group",
    var_name="year",
    value_name="gdp"
)

df_g["year"] = df_g["year"].replace({"gdp_2025": "2025", "gdp_2035": "2035"})

fig = px.bar(
    df_g,
    x="income_group",
    y="gdp",
    color="year",
    barmode="group",
    text="gdp",
    title="GDP per Capita (2025 vs 2035)"
)

fig.update_traces(texttemplate="%{text:.2f}", textposition="outside")
fig.show()


In [48]:
fig = px.scatter(
    final,
    x="mys_2035",
    y="gdp_2035",
    size="population_2035",
    color="income_group",
    text="income_group",
    title="Human Capital and Economic Outcomes (2035)",
    labels={
        "mys_2035": "Mean Years of Schooling (2035)",
        "gdp_2035": "GDP per Capita (2035)"
    },
    size_max=60
)

fig.update_traces(textposition="top center")
fig.show()
